In [4]:
!pip install catboost
!pip install optuna
!pip install dill
!pip install psutil
!pip install scikit-learn


  Using cached scikit_learn-1.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.9 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [7]:
from catboost import CatBoostClassifier
import numpy as np
import time
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

categories = ['class', 'quality', 'bathroom', 'bedding', 'capacity', 'club', 'bedrooms', 'balcony', 'view', 'floor']
models_dir = 'models_20240929_132912'


In [25]:
#! THIS CODE DOES NOT WORK. CREATES BAD NPY FILES

df = pd.read_csv('rates_clean.csv')
df = df.fillna('undefined')


for cat in categories:
    df[cat] = df[cat].astype(str)

# Convert 'rate_name' to string type
df['rate_name'] = df['rate_name'].astype(str)

X = df[['rate_name']]  # Keep as DataFrame
y = df[categories]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label_encoders = {}
for category in categories:
    le = LabelEncoder()
    y_train[category] = le.fit_transform(y_train[category])
    y_test[category] = le.transform(y_test[category])
    label_encoders[category] = le

for category in categories:
    np.save(os.path.join(models_dir, f"label_encoder_{category}.npy"), le.classes_)

FileNotFoundError: [Errno 2] No such file or directory: 'models_20240929_132912/label_encoder_class.npy'

# RUN PREDICTION

In [90]:
models = {}
label_encoders = {}

for category in categories:
    model_path = os.path.join(models_dir, f"catboost_model_{category}.cbm")
    model = CatBoostClassifier()
    model.load_model(model_path)
    models[category] = model

    le = LabelEncoder()
    le.classes_ = np.load(os.path.join(models_dir, f"label_encoder_{category}.npy"), allow_pickle=True)
    label_encoders[category] = le


def preprocess_input(input_strings):
    # Convert input strings to a DataFrame
    df = pd.DataFrame({'rate_name': input_strings})
    
    # Apply the same preprocessing as during training
    df['rate_name'] = df['rate_name'].astype(str)
    
    # Return the preprocessed DataFrame
    return df

def run_batch_prediction(model, label_encoder, input_strings):
    # Preprocess the input
    input_data = preprocess_input(input_strings)
    
    # Predict probabilities for the entire batch
    predictions = model.predict_proba(input_data)
    
    # Get the indices of the highest probabilities
    predicted_classes = predictions.argmax(axis=1)
    
    # Use the indices directly with classes_ for faster inverse transform
    return label_encoder.classes_[predicted_classes]

def classify_rates(rate_names: list) -> list:
    classifications = {category: run_batch_prediction(models[category], label_encoders[category], rate_names) 
                       for category in categories}
    return [dict(zip(categories, [classifications[cat][i] for cat in categories])) 
            for i in range(len(rate_names))]

df_dirty = pd.read_csv('rates_dirty.csv')
df_dirty = df_dirty.fillna('undefined')


batch_size = 6000  # нет смысла ставить больше 6000, так как после 5500 маленький прирост
results = []
benchmark_results = []
total_time = 0

for i in range(0, len(df_dirty), batch_size):
    batch = df_dirty.iloc[i:i+batch_size]
    rate_names = batch['rate_name'].tolist()
    
    iteration_start_time = time.time()
    batch_classifications = classify_rates(rate_names)
    iteration_end_time = time.time()
    
    iteration_time = iteration_end_time - iteration_start_time
    total_time += iteration_time
    benchmark_results.append({
        'batch_size': len(batch),
        'iteration_time': iteration_time,
    })
    
    for rate_name, classifications in zip(rate_names, batch_classifications):
        results.append({
            'rate_name': rate_name,
            **classifications
        })

print(f"Total time taken: {total_time:.10f} seconds")


df_results = pd.DataFrame(results)
df_benchmark = pd.DataFrame(benchmark_results)

avg_iteration_time = df_benchmark['iteration_time'].mean()
max_iteration_time = df_benchmark['iteration_time'].max()
min_iteration_time = df_benchmark['iteration_time'].min()

print(f"Batch execution time:")
print(f"Average iteration time: {avg_iteration_time * 1000:.4f} ms")
print(f"Maximum iteration time: {max_iteration_time * 1000:.4f} ms")
print(f"Minimum iteration time: {min_iteration_time * 1000:.4f} ms")
print(f"Avg iterationtime per 1 prediction: {avg_iteration_time / batch_size * 1e6:.4f} µs")

output_file = 'classified_rates.csv'
df_results.to_csv(output_file, index=False)

print(f"\nResults have been saved to {output_file}")

print("\nFirst few rows of the results:")
print(df_results[['rate_name'] + categories].head())

Total time taken: 1.6676299572 seconds
Batch execution time:
Average iteration time: 53.7945 ms
Maximum iteration time: 73.3070 ms
Minimum iteration time: 21.0950 ms
Avg iterationtime per 1 prediction: 8.9658 µs

Results have been saved to classified_rates.csv

First few rows of the results:
                                   rate_name  class    quality  \
0    Standard - Double Room - With Breakfast   room   standard   
1  6 Bed Non AC Mixed Dorm (1 Twin Bunk Bed)   dorm  undefined   
2             Standard Twin Room non-smoking   room   standard   
3    Deluxe Suite, Single or Double/Twin Bed  suite     deluxe   
4                 King Premium Mountain View   room    premium   

           bathroom                bedding   capacity      club   bedrooms  \
0  private bathroom  double/double-or-twin     double  not club  undefined   
1   shared bathroom              undefined     double  not club  undefined   
2  private bathroom    twin/twin-or-double     double  not club  undefined  

In [74]:
import csv
from difflib import SequenceMatcher
from collections import defaultdict

def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def compare_csv_files(file1_path, file2_path, threshold=0.8):
    with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
        reader1 = csv.reader(file1)
        reader2 = csv.reader(file2)
        
        header1 = next(reader1)
        header2 = next(reader2)
        
        if header1 != header2:
            print("Warning: Headers do not match")
        
        total_rows = 0
        error_rows = 0
        column_errors = defaultdict(int)
        
        for row1, row2 in zip(reader1, reader2):
            # Replace "undefined" with empty string
            row1 = ['' if val == 'undefined' else val for val in row1]
            row2 = ['' if val == 'undefined' else val for val in row2]

            total_rows += 1
            row_errors = 0
            
            for i, (val1, val2) in enumerate(zip(row1, row2)):
                similarity = calculate_similarity(val1, val2)
                if similarity < threshold:
                    row_errors += 1
                    column_errors[i] += 1
            
            error_rate = row_errors / len(row1)
            if error_rate > 0:
                error_rows += 1
                # print(f"Row {total_rows}: Error rate = {error_rate:.2f}")
                # print(f"Row {total_rows}: {row1} - {row2}")
        
        overall_error_rate = error_rows / total_rows
        print(f"\nOverall error rate: {overall_error_rate:.2f}")
        print(f"Total rows: {total_rows}")
        print(f"Rows with errors: {error_rows}")
        
        print("\nColumn Error Statistics:")
        for i, error_count in column_errors.items():
            column_error_rate = error_count / total_rows
            print(f"Column {i} ({header1[i]}): Error rate = {column_error_rate:.2f}, Errors = {error_count}")

# Usage
file1_path = 'classified_rates.csv'
file2_path = 'rates_dirty.csv'
compare_csv_files(file1_path, file2_path)


Overall error rate: 0.15
Total rows: 180443
Rows with errors: 26405

Column Error Statistics:
Column 5 (capacity): Error rate = 0.07, Errors = 12260
Column 9 (view): Error rate = 0.04, Errors = 6419
Column 3 (bathroom): Error rate = 0.01, Errors = 2700
Column 2 (quality): Error rate = 0.04, Errors = 7434
Column 1 (class): Error rate = 0.02, Errors = 4011
Column 4 (bedding): Error rate = 0.05, Errors = 9139
Column 6 (club): Error rate = 0.01, Errors = 1907
Column 7 (bedrooms): Error rate = 0.01, Errors = 2439
Column 10 (floor): Error rate = 0.01, Errors = 2138
